In [24]:
from pxr import Usd,Sdf

In [25]:
stage = Usd.Stage.Open(r"D:\Kitchen_set\Kitchen_set.usd")
stage.Reload()
layer = stage.GetSessionLayer()


In [26]:
# テスト用の編集をSessionLayerに書く
target = stage.GetEditTargetForLocalLayer(layer)
stage.SetEditTarget(target)

In [36]:
# Collectionを作る
collection = stage.DefinePrim("/collectionSample")
prim = stage.GetPrimAtPath("/Kitchen_set/Props_grp/West_grp")
collectionName = "sampleCollection"
api = Usd.CollectionAPI.Apply(collection,collectionName)
print(api.GetName())

sampleCollection


In [28]:
# WestGrp以下をCollectionに入れる
api.IncludePath(prim.GetPath())
# ただしそのなかの SteamCooker_1 は除外
api.ExcludePath(Sdf.Path("/Kitchen_set/Props_grp/West_grp/SteamCooker_1"))

True

In [29]:
# ExpansionRuleとは、Collectionに入れているPrimの展開方法（Prim以下すべてなのかそれだけなのか）などを指定するもの。
expansionRule = api.CreateExpansionRuleAttr()
print(expansionRule)
# 取得
print(api.GetExpansionRuleAttr())

print(expansionRule.Get())
# explicitOnly/expandPrims/expandPrimsAndProperties
#expansionRule.Set("expandPrims")
expansionRule.Set("explicitOnly")
api.IncludePath("/Kitchen_set/Props_grp/West_grp/Ball_1")

Usd.Prim(</collectionSample>).GetAttribute('collection:sampleCollection:expansionRule')
Usd.Prim(</collectionSample>).GetAttribute('collection:sampleCollection:expansionRule')
expandPrims


True

In [30]:
# 条件に見合うPrimをStageから探す
query = api.ComputeMembershipQuery()
# emplicitOnlyだと、IncludePathに入れたPrimのみが検索対象
print(Usd.CollectionAPI.ComputeIncludedObjects(query,stage))

[Usd.Prim(</Kitchen_set/Props_grp/West_grp>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/Ball_1>)]


In [33]:
# ExpansionRuleを変えてみる
expansionRule.Set("expandPrims")
queryB = api.ComputeMembershipQuery()
# expandPrimsで指定されてるので、IncludePath以下にありExcludeに含まれないPrimがリストされる
print(Usd.CollectionAPI.ComputeIncludedObjects(queryB,stage))

[Usd.Prim(</Kitchen_set/Props_grp/West_grp>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/Ball_1>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/Ball_1/Geom>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/Ball_1/Geom/Ball>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/BottleTall_1>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/BottleTall_1/Geom>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/BottleTall_1/Geom/Bottle>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_grp>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_grp/BookBlue_1>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_grp/BookBlue_1/Geom>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_grp/BookBlue_1/Geom/Book>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_grp/BookBlue_1/Geom/pCube134>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_grp/BookGreen_1>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_grp/BookGreen_1/Geom>), Usd.Prim(</Kitchen_set/Props_grp/West_grp/FoldingTable_g

In [35]:
# 引数で指定したSdfPathがCollectionに含まれるか
print(queryB.IsPathIncluded("/Kitchen_set/Props_grp/West_grp/ShellLarge_1"))
# Collectionより親のPrim
print(queryB.IsPathIncluded("/Kitchen_set"))
# Excludeに指定されているPrim以下
print(queryB.IsPathIncluded("/Kitchen_set/Props_grp/West_grp/SteamCooker_1/Geom/pCylinder55"))

True
False
False


In [10]:
print(expansionRule.Get())

expandPrims


In [11]:
# CollectionにセットされているPrimを取得
# CollectionはRelationで指定されている
print(api.GetExcludesRel().GetTargets())
print(api.GetIncludesRel().GetTargets())

[Sdf.Path('/Kitchen_set/Props_grp/West_grp/SteamCooker_1')]
[Sdf.Path('/Kitchen_set/Props_grp/West_grp'), Sdf.Path('/Kitchen_set/Props_grp/West_grp/Ball_1')]


In [12]:
# どういうルールでQueryされるか確認
print(query.GetAsPathExpansionRuleMap())

{Sdf.Path('/Kitchen_set/Props_grp/West_grp'): 'explicitOnly', Sdf.Path('/Kitchen_set/Props_grp/West_grp/Ball_1'): 'explicitOnly', Sdf.Path('/Kitchen_set/Props_grp/West_grp/SteamCooker_1'): 'exclude'}


In [13]:
# APIで指定されているCollectionのPath
print(api.GetCollectionPath())

/collectionSample.collection:sampleCollection


In [37]:
# MultipleApplyのテスト
# とりあえず空のPrimを作る。もちろんCollectionAPIは適応されていない。
collectionB = stage.DefinePrim("/collectionB")
print(collection.HasAPI(Usd.CollectionAPI))
print(collectionB.HasAPI(Usd.CollectionAPI))

True
False


In [15]:
# 同じPrimに対して別のCollectionを指定
api.Apply(collectionB,'sampleCollectionA')

Usd.CollectionAPI(Usd.Prim(</collectionB>), 'sampleCollectionA')

In [16]:
# Primに対してAPIがあるかどうかをチェック
stage.GetPrimAtPath("/collectionB").HasAPI(Usd.CollectionAPI)

True

In [22]:
# 結果を確認
print(layer.ExportToString())

#usda 1.0

def "collectionSample" (
    prepend apiSchemas = ["CollectionAPI:collectionName"]
)
{
    prepend rel collection:sampleCollection:excludes = </Kitchen_set/Props_grp/West_grp/SteamCooker_1>
    uniform token collection:sampleCollection:expansionRule = "expandPrims"
    prepend rel collection:sampleCollection:includes = [
        </Kitchen_set/Props_grp/West_grp>,
        </Kitchen_set/Props_grp/West_grp/Ball_1>,
    ]
}

def "collectionB" (
    prepend apiSchemas = ["CollectionAPI:sampleCollectionA"]
)
{
}


